In [1]:
import pandas as pd
melb_data = pd.read_csv('C:/Users/U_M1P8G/Desktop/IDE/Python_for_data_analysis/Модуль_3/data/melb_data.csv', sep=',')
melb_df = melb_data.copy()

In [2]:
print(melb_df['Address'].nunique())

13378


In [3]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [5]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

In [6]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [8]:
print(street_types.nunique())
display(street_types.value_counts())

56


Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Grn             4
Fairway         4
Mews            4
Gdns            4
Righi           3
Crossway        3
Esp             2
Ridge           2
Victoria        2
Crofts          2
Athol           1
Highway         1
Cove            1
Grange          1
Res             1
Terrace         1
Qy              1
Glade           1
Nook            1
Eyrie           1
Loop            1
Dell            1
East            1
Summit          1
Grand           1
Gra             1
Hts             1
Outlook         1
Woodland        1
Ave             1
Co

К результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:

In [9]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


Введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [11]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [14]:
print(melb_df['StreetType'].nunique())

11


In [15]:
melb_df = melb_df.drop('Address', axis=1)

In [17]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale']=melb_df['Date'].dt.dayofweek 
melb_df.value_counts('WeekdaySale')
weekend_count = melb_df[melb_df['WeekdaySale'].isin([5, 6])].shape[0]
display(melb_df['WeekdaySale'])


0        5
1        3
2        5
3        5
4        5
        ..
13575    5
13576    5
13577    5
13578    5
13579    5
Name: WeekdaySale, Length: 13580, dtype: int32

Задание 4.2

In [22]:
def get_weekend(weekday):
    if weekday in [5, 6]:
        return 1
    else:
        return 0
melb_df['weekday_func'] = melb_df['WeekdaySale'].apply(get_weekend)   
melb_df[melb_df['weekday_func']==1]['Price'].mean()

np.float64(1081198.6406956792)

Задание 4.3

In [24]:
popular_sellers =melb_df['SellerG'].value_counts().nlargest(49).index
print(popular_sellers)

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Eview', 'Purplebricks', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'McDonald', 'Burnham',
       'Moonee', 'LITTLE'],
      dtype='object', name='SellerG')


In [25]:
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
display(melb_df['SellerG'])

0          Biggin
1          Biggin
2          Biggin
3          Biggin
4          Nelson
           ...   
13575       Barry
13576    Williams
13577       Raine
13578     Sweeney
13579     Village
Name: SellerG, Length: 13580, dtype: object

In [29]:
Nelson_price = melb_df[melb_df['SellerG']=='Nelson']['Price'].min()
Other_price = melb_df[melb_df['SellerG']=='other']['Price'].min()
answer = Nelson_price/Other_price
answer

np.float64(1.297709923664122)